# CLASSES & OOP

## Base Class

Additional topics include:
* __OOP: Encapsultation__ (public & private acceses)
    * for more info on access keywords, see [here](https://learn.microsoft.com/en-us/dotnet/csharp/tour-of-csharp/program-building-blocks#accessibility)
* __OOP: Abstraction__
    * NOTE: BankAccount class is seen as an abstract, base class that will be inherited by more specific bank account classes below (e.g. InterestEarningAccount, LineOfCreditAccount, GiftCardAccount)
* Getters & setters
* Constructor
* Exception handling & raising
* `virtual` methods (and brief on `abstract` methods)
* StringBuilder objects (kinda like Python's 3-quoted strings or .join())

In [1]:
public class Transaction {
    public decimal Amount { get; }
    public DateTime Date { get; }
    public string Description { get; }

    public Transaction(decimal amount, DateTime date, string description) {
        this.Amount = amount;
        this.Date = date;
        this.Description = description;
    }
}



// CENTRAL CLASS
public class BankAccount {
    // Shorthand getter & setter methods, see https://codeeasy.io/lesson/properties
    public string ID { get; }
    public string Owner { get; set; }
    public decimal Balance { 
        get { // custom getter
            decimal balance = 0;

            foreach (var transaction in allTransactions) {
                balance += transaction.Amount;
            }

            return balance;
        } 
    }

    private static int accountNumberSeed = 101;
    
    private List<Transaction> allTransactions = new List<Transaction>();

    // Constructor
    public BankAccount(string name, decimal initialBalance) {
        this.Owner = name;
        // no need to assign 'this.Balance' in constructor since a custom getter has been written above
        // this.Balance = initialBalance; 
        this.MakeDeposit(initialBalance, DateTime.Now, "Initial Balance");
        this.ID = accountNumberSeed.ToString();
        accountNumberSeed++;
    }

    public void MakeDeposit(decimal amount, DateTime date, string description) {
        if (amount <= 0) {
            // raising/throwing exceptions
            throw new ArgumentOutOfRangeException(nameof(amount), "Amount of deposit must be positive");
        }
        
        var deposit = new Transaction(amount, date, description);
        allTransactions.Add(deposit);
    }

    public void MakeWithdrawal(decimal amount, DateTime date, string description) {
        if (Balance - amount < 0) {
            throw new InvalidOperationException("You're broke INSUFFICIENT FUNDS");
        }
        var withdrawal = new Transaction(-amount, date, description);
        allTransactions.Add(withdrawal);
    }

    // Will be used to demonstrate polymorphism in InterestEarningAccount class below
    // `virtual` keyword gives inherited classes the (optional) ability to override base class implementation
    // an `abstract` keyword is similar to `virtual`, but it is required to override base class implementation
    public virtual void PerformMonthEndTransactions() { }


    // Using a StringBuilder object to get transaction history
    // StringBuilders are kinda like Python's 3-quoted strings or .join()
    public string GetTransactionStatement() {
        var report = new StringBuilder(); // in System.Text 

        //HEADER
        report.AppendLine("Date\t\tAmount\tDescription"); // '\t' is tabbed whitespace

        foreach (var transaction in allTransactions) {
            //ROWS
            // DateTime.ToShortDateString() excludes the time and only includes the date
            report.AppendLine($"{transaction.Date.ToShortDateString()}\t${transaction.Amount}\t{transaction.Description}");
        }

        return report.ToString();
    }
}


### *just pretend that the cells below are the Main method for now*

In [2]:
var account1 = new BankAccount("Squidward", 260);
Console.WriteLine($"New bank account for {account1.Owner} [Account ID #{account1.ID}], starting with ${account1.Balance}");

New bank account for Squidward [Account ID #101], starting with $260


In [3]:
account1.MakeWithdrawal(75, DateTime.Now, "Clarinet");
Console.WriteLine($"$75 has been debited from {account1.Owner}'s account. Remaining balance: ${account1.Balance}");

$75 has been debited from Squidward's account. Remaining balance: $185


Handling exceptions (try/catch):

In [4]:
try{
    account1.MakeDeposit(-300, DateTime.Now, "tax fraud");
}
catch (ArgumentOutOfRangeException e){
    Console.WriteLine("Attempting to deposit a negative amount, cancelling transaction...");
    Console.WriteLine($"Exception is as follows: {e.ToString()}");
}

Attempting to deposit a negative amount, cancelling transaction...
Exception is as follows: System.ArgumentOutOfRangeException: Amount of deposit must be positive (Parameter 'amount')
   at Submission#2.BankAccount.MakeDeposit(Decimal amount, DateTime date, String description)
   at Submission#5.<<Initialize>>d__0.MoveNext()


In [5]:
try{
    account1.MakeWithdrawal(9175, DateTime.Now, "Rent");
    Console.WriteLine($"$9175 has been debited from {account1.Owner}'s account. Remaining balance: ${account1.Balance}");
}
catch (InvalidOperationException e){
    Console.WriteLine("You're broke INSUFFICIENT FUNDS");
    Console.WriteLine($"Exception is as follows: {e.ToString()}");
}

You're broke INSUFFICIENT FUNDS
Exception is as follows: System.InvalidOperationException: You're broke INSUFFICIENT FUNDS
   at Submission#2.BankAccount.MakeWithdrawal(Decimal amount, DateTime date, String description)
   at Submission#6.<<Initialize>>d__0.MoveNext()


Using method that uses StringBuilder object to get print out of all transaction history:

In [6]:
Console.WriteLine(account1.GetTransactionStatement());

Date		Amount	Description
4/30/2023	$260	Initial Balance
4/30/2023	$-75	Clarinet



<hr>

## Inheritance, Polymorphism, & Overrides

Additional topics include:
* Inherited/derived class constructors
* Optional/default parameters

In [7]:
public class InterestEarningAccount : BankAccount {

    // If constructor is defined in base class, derived class must also declare a constructor that includes all parameters
    // from base class constructor
    public InterestEarningAccount(string name, decimal initialBalance) : base(name, initialBalance) { }

    // derived from virtual method in BankAccount
    public override void PerformMonthEndTransactions() {
        if (Balance > 500m)
        {
            decimal interest = Balance * 0.05m;
            MakeDeposit(interest, DateTime.Now, "apply monthly interest");
        }
    }
}




public class LineOfCreditAccount : BankAccount {
    public LineOfCreditAccount(string name, decimal initialBalance) : base(name, initialBalance) { }

    // derived from virtual method in BankAccount
    public override void PerformMonthEndTransactions()
    {
        if (Balance < 0)
        {
            // Negate the balance to get a positive interest charge:
            decimal interest = -Balance * 0.07m;
            MakeWithdrawal(interest, DateTime.Now, "Charge monthly interest");
        }
    }
}




public class GiftCardAccount : BankAccount {
    private readonly decimal _monthlyDeposit = 0m;

    // optional/default parameter: monthlyDeposit
    public GiftCardAccount(string name, decimal initialBalance, decimal monthlyDeposit = 0) : base(name, initialBalance)
        => _monthlyDeposit = monthlyDeposit;

    public override void PerformMonthEndTransactions()
    {
        if (_monthlyDeposit != 0)
        {
            MakeDeposit(_monthlyDeposit, DateTime.Now, "Add monthly deposit");
        }
    }
    
}

In [9]:
var giftCard = new GiftCardAccount("gift card", 100, 50);
giftCard.MakeWithdrawal(20, DateTime.Now, "get expensive coffee");
giftCard.MakeWithdrawal(50, DateTime.Now, "buy groceries");
giftCard.PerformMonthEndTransactions();
// can make additional deposits:
giftCard.MakeDeposit(27.50m, DateTime.Now, "add some additional spending money");
Console.WriteLine(giftCard.GetTransactionStatement());

var savings = new InterestEarningAccount("savings account", 10000);
savings.MakeDeposit(750, DateTime.Now, "save some money");
savings.MakeDeposit(1250, DateTime.Now, "Add more savings");
savings.MakeWithdrawal(250, DateTime.Now, "Needed to pay monthly bills");
savings.PerformMonthEndTransactions();
Console.WriteLine(savings.GetTransactionStatement());

Date		Amount	Description
4/30/2023	$100	Initial Balance
4/30/2023	$-20	get expensive coffee
4/30/2023	$-50	buy groceries
4/30/2023	$50	Add monthly deposit
4/30/2023	$27.50	add some additional spending money

Date		Amount	Description
4/30/2023	$10000	Initial Balance
4/30/2023	$750	save some money
4/30/2023	$1250	Add more savings
4/30/2023	$-250	Needed to pay monthly bills
4/30/2023	$587.50	apply monthly interest

